# Introduction
Models are trained in google colab with batch_size of 200-700. For online prediction, we only need batch_size of 1.
Here we recombined models in 

### Get source folder and append to sys directory

In [1]:
from __future__ import print_function
import os
import sys
PROJ_ROOT = os.path.join(os.pardir)
print(os.path.abspath(PROJ_ROOT))
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)
# Data path example
#pump_data_path = os.path.join(PROJ_ROOT,
#                              "data",
#                              "raw",
#                              "pumps_train_values.csv")

/mnt/4ba37af6-51fd-47bc-8321-8c500c229114/study/School/KHOA LUAN TOT NGHIEP/runnable_program


### Imports
Import libraries and write settings here.

In [2]:
# Data manipulation
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn
import random

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython import get_ipython
ipython = get_ipython()
# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 1
# Use %aimport module to reload each module
%aimport data.read_data
from data.read_data import read_data_by_type
from data.create_load_transform_processed_data import load_reshaped_array, create_tensorflow_dataset
# Visualizations
import matplotlib.pyplot as plt

In [3]:
data = read_data_by_type(PROJ_ROOT=PROJ_ROOT,data_type="interim", output_type="dataframe")
data

CO  NO2  PM25  AQI_h AQI_h_Polutant  AQI_h_I  \
site_id time                                                                 
30      2020-12-05 00:00:00  2.0  7.0  11.0    7.0            NO2      1.0   
        2020-12-05 01:00:00  1.0  6.0  11.0   11.0           PM25      1.0   
        2020-12-05 02:00:00  1.0  6.0  13.0   13.0           PM25      1.0   
        2020-12-05 03:00:00  1.0  6.0  18.0   18.0           PM25      1.0   
        2020-12-05 04:00:00  1.0  7.0  17.0   17.0           PM25      1.0   
...                          ...  ...   ...    ...            ...      ...   
36      2021-06-23 11:00:00  5.0  8.0  16.0   16.0           PM25      1.0   
        2021-06-23 12:00:00  5.0  6.0  14.0   14.0           PM25      1.0   
        2021-06-23 13:00:00  3.0  6.0   8.0    8.0           PM25      1.0   
        2021-06-23 14:00:00  2.0  7.0   5.0    7.0            NO2      1.0   
        2021-06-23 15:00:00  3.0  7.0   3.0    7.0            NO2      1.0   

                            AQI_h_label  
site_id time                             
30      2020-12-05 00:00:00        Good  
        2020-12-05 01:00:00        Good  
        2020-12-05 02:00:00        Good  
        2020-12-05 03:00:00        Good  
        2020-12-05 04:00:00        Good  
...                                 ...  
36      2021-06-23 11:00:00        Good  
        2021-06-23 12:00:00        Good  
        2021-06-23 13:00:00        Good  
        2021-06-23 14:00:00        Good  
        2021-06-23 15:00:00        Good  

[158104 rows x 7 columns]

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))
scaler_PM25 = MinMaxScaler(feature_range=(-1,1))
scaler_PM25.fit(data['PM25'].values.reshape(-1, 1))
scaler_AQI = MinMaxScaler(feature_range=(-1,1))
scaler_AQI.fit(data['AQI_h'].values.reshape(-1, 1))

MinMaxScaler(feature_range=(-1, 1))

MinMaxScaler(feature_range=(-1, 1))

# Analysis/Modeling

In [5]:
# What I need to do: Read all models and their weights
# save them as a new model with included weights
# Check the performance of saved models with loaded models

In [7]:
from models.create_and_load_model import create_model
# Read models and combine them with weights
_data_to_model_path = os.path.join(PROJ_ROOT,
                                    "data",
                                    "model_input",
                                    "hanoi")
_hanoi_model_path = os.path.join(PROJ_ROOT,
                            "models",
                            "hcm")
_converted_model_path = os.path.join(PROJ_ROOT,
                                    "models",
                                    "combined")

def get_model_name(timesteps, target_hour):
    return 'model_{}_{}.h5'.format(timesteps, target_hour)

def get_saved_model_name(timesteps, target_hour):
    # Saved model but for gcp
    return 'model_{}_{}'.format(timesteps, target_hour)

def get_model_weigts_name(timesteps, target_hour):
    return 'weights_{}_{}.ckpt'.format(timesteps, target_hour)
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0))
def mean_absolute_percentage_error(y_true, y_pred,
                                   sample_weight=None,
                                   multioutput='uniform_average'):
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape,
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)

for hour in [1]:
    rmse = []
    r2 = []
    mae = []
    mape = []
    for timesteps in [5]:
        print(timesteps)
        batch_size = 700
        test, y_test = load_reshaped_array(timesteps, target_hour=hour, folder_path=_data_to_model_path, data_type="test")
        test_data_tf, test_steps_per_epochs = create_tensorflow_dataset(test, y_test, batch_size)
        
        colab_model = keras.models.load_model("{}/{}".format(_hanoi_model_path,get_model_name(timesteps=timesteps, target_hour=hour)),
                                custom_objects={'LeakyReLU': layers.LeakyReLU(alpha=0.01),
                                               'root_mean_squared_error': root_mean_squared_error})
        old_input_layer = colab_model.layers.pop(0)
        timesteps, features = old_input_layer.output_shape[0][1:]
        # Model with batch_size=1
        converted_model = create_model(batch_size=1, timesteps=timesteps, features=features)
        converted_model.load_weights('{}/{}'.format(_hanoi_model_path,
                                         get_model_weigts_name(timesteps=timesteps, target_hour=hour))).expect_partial()
        converted_model.save("{}/{}".format(_converted_model_path, get_saved_model_name(timesteps=timesteps, target_hour=hour)))

        # np.testing.assert_allclose(
        #     model.predict(test_data_tf, steps=test_steps_per_epochs), reconstructed_model.predict(test_data_tf, steps=test_steps_per_epochs),

        # )
        # print(model.predict(test_data_tf, steps=test_steps_per_epochs))

#         scaler_AQI = MinMaxScaler(feature_range=(-1,1))
#         scaler_AQI.fit(thudohanoi_df['AQI_h'].values.reshape(-1, 1))
#         y_test = scaler_AQI.inverse_transform(y_test.reshape(-1, 1))
#         predict = scaler_AQI.inverse_transform(predict.reshape(-1, 1))
#         print("=============================================\n")
# #         print("Predict")
# #         predict_vs_truth = pd.DataFrame({'predict': predict[rand:rand+20],
# #                                         'truth': y_test[rand:rand+20]})
# #         print(predict_vs_truth)
#         print("R2: {}".format(r2_score(predict, y_test)))
#         print("Root mean squared error: {}".format(mean_squared_error(predict, y_test, squared=False)))
#         print("Mean absolute percentage error: {}".format(mean_absolute_percentage_error(predict, y_test)))
#         print("Mean absolute error: {}".format(mean_absolute_error(predict, y_test)))
        
#         rmse.append(mean_squared_error(predict, y_test, squared=False))
#         r2.append(r2_score(predict, y_test))
#         mape.append(mean_absolute_percentage_error(predict, y_test))
#         mae.append(mean_absolute_error(predict, y_test))

5
[[[-0.76        0.          0.         ... -0.55393939 -0.55022222
   -0.46533333]
  [-0.78666667  0.          0.         ... -0.57672727 -0.57111111
   -0.48155556]
  [-0.79733333  0.          0.         ... -0.6009697  -0.59422222
   -0.49933333]
  [-0.94133333  0.          0.         ... -0.62472727 -0.61733333
   -0.52288889]
  [-0.86666667  0.          0.         ... -0.66157576 -0.65111111
   -0.55111111]]

 [[-0.78666667  0.          0.         ... -0.57672727 -0.57111111
   -0.48155556]
  [-0.79733333  0.          0.         ... -0.6009697  -0.59422222
   -0.49933333]
  [-0.94133333  0.          0.         ... -0.62472727 -0.61733333
   -0.52288889]
  [-0.86666667  0.          0.         ... -0.66157576 -0.65111111
   -0.55111111]
  [-0.824       0.          0.         ... -0.69163636 -0.67866667
   -0.57466667]]

 [[-0.79733333  0.          0.         ... -0.6009697  -0.59422222
   -0.49933333]
  [-0.94133333  0.          0.         ... -0.62472727 -0.61733333
   -0.52288889

INFO:tensorflow:Assets written to: ../models/combined/model_5_1/assets
INFO:tensorflow:Assets written to: ../models/combined/model_5_1/assets


In [12]:
# Try predict with converted models
converted_models.predict(test_data_tf, steps=test_steps_per_epochs)

array([[-1.0194583],
       [-1.0246565],
       [-1.0255059],
       ...,
       [-1.0540177],
       [-1.0577075],
       [-1.0468792]], dtype=float32)

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here